## Overview
This notebook captures the implementation of training an image classification model to identify if a crop is infected based on the input leaf image. This notebook will be executed in kaggle because of large data size as well as the various options we would like to explore for improving the training accuracy 
- Different Learning Rates : Warm-up LR, Step Decay LR, Cosine Decay LR, Hybrid, Annealing, Adaptive Decay LR
- Different Cost Functions : Use Focal Loss function
- Label Smoothening : Instead of a 0 and 1 for the labels create a continuous distribution of the output

In [ ]:
# !pip install torch_summary

In [1]:
import os
import torch
import warnings
warnings.simplefilter('ignore')
import numpy as np
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models

import imgaug as ia
import imgaug.augmenters as iaa
from collections import Counter
from torchsummary import summary

In [2]:
TRAIN_DIR = '../input/cropdata/input/train/'
TEST_DIR = '../input/cropdata/input/train/'
VALIDATE_DIR = '../input/cropdata/input/train/'
BATCH_SIZE = 128

#### For GPU

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data

In [4]:
class CropDataset(Dataset):
    def __init__(self, file_path, transform=None, augment=False):
        xform = transforms.Compose([transforms.ToTensor()]) if transform is None else transform
        self.img_dataset = datasets.ImageFolder(file_path, transform=xform)
        self.aug = augment
        
        ## augumentation initialization
        ia.seed(241)

        self.seq = iaa.Sequential([
            iaa.Fliplr(0.5), # horizontal flips
            iaa.Crop(percent=(0, 0.1)), # random crops
            # Small gaussian blur with random sigma between 0 and 0.5.
            # But we only blur about 50% of all images.
            iaa.Sometimes(
                0.5,
                iaa.GaussianBlur(sigma=(0, 0.5))
            ),
            # Strengthen or weaken the contrast in each image.
            iaa.LinearContrast((0.75, 1.5)),
                        
            # Make some images brighter and some darker.
            # In 20% of all cases, we sample the multiplier once per channel,
            # which can end up changing the color of the images.
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            # Apply affine transformations to each image.
            # Scale/zoom them, translate/move them, rotate them and shear them.
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-25, 25),
                shear=(-8, 8)
            )
        ], random_order=True) # apply augmenters in random order

    def __len__(self):
        return len(self.img_dataset)

    def __getitem__(self, idx):
        image, label = self.img_dataset[idx]
        return image, label
    
    # this is an experimental method created to explore how the dunder methods can be modified
    def __repr__(self, idx=0):
        image, label = self.img_dataset[idx]
        plt.imshow(image.permute(1,2,0))
        return str("Display view of the image")
    
    def collate_fn(self, batch):
        images, targets = list(zip(*batch))
        images = torch.stack(images).permute(0 , 2, 3, 1)
        if self.aug: images=self.seq.augment_images(images=images.numpy()) 
        targets = torch.tensor(targets)
        images = torch.tensor(images).permute(0, 3, 1, 2)
        if torch.cuda.is_available():
            targets = targets.to(device)
            images = images.to(device)
        return images, targets

In [5]:
# Here we create our final transformation that would be used before we send the data for training the model
transform = torchvision.transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize((0.4743617, 0.49847862, 0.4265874 ),
                                                                 (0.21134755, 0.19044809, 0.22679578))]
                                          )
crop_dataset = CropDataset(TRAIN_DIR, transform, True)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Peach___Bacterial_spot': 15, 'Peach___healthy': 16, 'Pepper,_bell___Bacterial_spot': 17, 'Pepper,_bell___healthy': 18, 'Potato___Early_blight': 19, 'Potato___Late_blight': 20, 'Potato___healthy': 21, 'Strawberry___Leaf_scorch': 22, 'Strawberry___healthy': 23, 'Tomato___Bacterial_spot': 24, 'Tomato___Early_blight': 25, 'Tomato___Late_blight': 26, 'Tomato___Leaf_Mold': 27, 'Tomato___Septoria_leaf_spot': 28, 'Tomato___Spider_mites Two-spotted_spider_mite': 29, 'Tomato___Target_Spot': 

In [6]:
# define a weighted sampler for the images 
num_samples = len(crop_dataset.img_dataset)
label = list(crop_dataset.img_dataset.targets)
class_weights = [round(1.0/v,5) for k, v in dict(Counter(crop_dataset.img_dataset.targets)).items()]
weights = [class_weights[label[i]] for i in range(num_samples)]

# Now we can create a loader that will help us load images in batches for training purpose 
sampler = WeightedRandomSampler(weights, num_samples, replacement=True)
crop_loader = DataLoader(dataset=crop_dataset.img_dataset, 
                         batch_size=BATCH_SIZE, 
                         sampler=sampler, 
                         #collate_fn=crop_dataset.collate_fn
                        )

In [8]:
lst_target = []
for idx, (image, target) in enumerate(crop_loader):
    lst_target.extend(list(target.numpy()))
    if idx%100 == 0 and idx > 0:
        print(f'Included data from first {idx * BATCH_SIZE} samples')
print(f'Included data from all {num_samples} samples')
print(f'Class distribution {dict(Counter(lst_target))}')
print(f'Total {len(lst_target)} records processed')

Included data from all 10939 samples
Class distribution {17: 328, 24: 327, 25: 371, 13: 337, 18: 297, 6: 325, 30: 303, 12: 327, 31: 309, 0: 283, 15: 307, 4: 331, 10: 335, 16: 312, 32: 326, 21: 339, 1: 334, 20: 306, 33: 323, 3: 313, 7: 331, 14: 326, 5: 312, 8: 322, 26: 319, 2: 304, 22: 346, 19: 322, 9: 346, 11: 306, 27: 328, 28: 316, 29: 328, 23: 300}
Total 10939 records processed


#### Test Data

In [ ]:
test_dataset = CropDataset(TEST_DIR, transform, True)

In [ ]:
test_loader = DataLoader(dataset=test_dataset.img_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle= True
                         #collate_fn=crop_dataset.collate_fn
                        )

#### Validation Data

In [ ]:
valid_dataset = CropDataset(VALIDATE_DIR, transform, True)

In [ ]:
valid_loader = DataLoader(dataset=valid_dataset.img_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle= True
                         #collate_fn=crop_dataset.collate_fn
                        )

## Initialize

In [ ]:
def train_model(model, n_epochs, data_loader):
    train_epochs, train_loss, train_accuracy = [], [], []
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        total = 0
        correct = 0
        model.train()
        for i, data in enumerate(data_loader, 0):
            if (torch.cuda.is_available()):
                inputs, labels = data[0].to(device), data[1].to(device)
            else:
                inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if i % 100 == 0 and i > 0:    
                accu=100.*correct/total
                mloss=running_loss / 100.
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {mloss:.3f} accuracy:{accu:.3f}')
                train_epochs.append(epoch + 1)
                train_loss.append(mloss)
                train_accuracy.append(accu)
                running_loss = 0.0
    print('Finished Training')
    return train_epochs, train_loss, train_accuracy

In [ ]:
def compute_accuracy(model, data_loader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct // total
    print(f'Accuracy of the network on the test images: {accuracy} %')
    return accuracy

In [ ]:
def compute_class_accuracy(model, dataset, data_loader):
    # prepare to count predictions for each class
    correct_pred = {value:0 for key, value in dataset.img_dataset.class_to_idx.items()}
    total_pred = {value:0 for key, value in dataset.img_dataset.class_to_idx.items()}

    # again no gradients needed
    with torch.no_grad():
        for data in data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[int(label)] += 1
                total_pred[int(label)] += 1

    accuracy_per_class = {}
    # print accuracy for each class
    for key, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[key]
        print(f'Accuracy for class: {key:5d} is {accuracy:.1f} %')
        accuracy_per_class[key] = accuracy
    
    return accuracy_per_class

## Model

### Custom Model

In [9]:
# Now create a a model that can be trained for disease detection
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 9)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 6)
        self.conv3 = nn.Conv2d(12, 18, 3)
        self.fc1 = nn.Linear(18 * 28 * 28, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 38)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
if(torch.cuda.is_available()):
    net.to(device)

In [ ]:
cust_epochs, cust_loss, cust_accuracy = train_model(net, 10, crop_loader)

In [ ]:
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline

plt.subplot(211)
plt.plot(cust_epochs, cust_loss, 'bo', label='Training loss')
plt.title('Training loss (Custom Model)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid('off')
plt.show()

plt.subplot(212)
plt.plot(cust_epochs, cust_accuracy, 'r', label='Training accuracy')
plt.title('Training accuracy (Custom Model)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid('off')
plt.show()

#### Test Custom Model

In [ ]:
cust_test_accuracy = compute_accuracy(net, test_loader)

In [ ]:
cust_test_accuracy_per_class = compute_class_accuracy(net, test_dataset, test_loader)

#### Validate Custom Model

In [ ]:
cust_valid_accuracy = compute_accuracy(net, valid_loader)

In [ ]:
cust_valid_accuracy_per_class = compute_class_accuracy(net, valid_dataset, valid_loader)

### Transfer Learning - VGG

In [ ]:
def vgg_model():
    
    # initialize a vgg model
    vgg_model = models.vgg16(pretrained=True)
    
    # freeze all the parameters in the sequentional layer
    for param in vgg_model.parameters():
        param.requires_grad = False
    
    # change the average pool layer
    vgg_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(7,7))

    # Change the Classifier layer
    vgg_model.classifier = nn.Sequential(nn.Flatten(),
                                    nn.Linear(25088, 4096),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(4096, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 34))
    return vgg_model

In [ ]:
vgg16 = vgg_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
if(torch.cuda.is_available()):
    vgg16 = vgg16.to(device)

In [ ]:
summary(vgg16, torch.zeros(1,3,224,224))

In [ ]:
vgg_epochs, vgg_loss, vgg_accuracy = train_model(vgg16, 5,crop_loader)

In [ ]:
plt.subplot(211)
plt.plot(vgg_epochs, vgg_loss, 'bo', label='Training loss')
plt.title('Training loss (VGG)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid('off')
plt.show()

plt.subplot(212)
plt.plot(vgg_epochs, vgg_accuracy, 'r', label='Training accuracy')
plt.title('Training accuracy (VGG)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid('off')
plt.show()

#### Test VGG

In [ ]:
vgg_test_accuracy = compute_accuracy(vgg16, test_loader)

In [ ]:
vgg_test_accuracy_per_class = compute_class_accuracy(vgg16, test_dataset, test_loader)

#### Validate VGG

In [ ]:
vgg_valid_accuracy= compute_accuracy(vgg16, valid_loader)

In [ ]:
vgg_valid_accuracy_per_class = compute_class_accuracy(vgg16, valid_dataset, valid_loader)

#### Save VGG

In [18]:
torch.save(vgg16, 'crop_disease_model_local.pth')

### Transfer Learning - ResNet

### Appendix

#### Distribution of Weights

In [ ]:
for ix, par in enumerate(net.parameters()):
    if(ix%2==0):
        plt.hist(par.cpu().detach().numpy().flatten())
        plt.title(f'Distribution of weights conencting layer \
{ix} to hidden layer {ix + 1}')
        plt.show()
    elif(ix%2==1):
        plt.hist(par.cpu().detach().numpy().flatten())
        plt.title(f'Distribution of biases of layer {ix}')
        plt.show()